In [1]:
import torch
from modules import SConv2d, SLinear, SModel, SReLU
from models import SCrossEntropyLoss

In [2]:
fc1 = SLinear(3,3,bias=False)
fc1.op.weight.data = torch.Tensor(list(range(1,10))).view(fc1.op.weight.shape) * 0.1
# fc1.op.bias.data = torch.zeros_like(fc1.op.bias)
fc2 = SLinear(3,3,bias=False)
fc2.op.weight.data = torch.Tensor(list(range(9,0,-1))).view(fc2.op.weight.shape) * 0.1
fc2.op.weight.data[0,0] = 2
# fc2.op.bias.data = torch.zeros_like(fc2.op.bias)
relu = SReLU()
criteria = SCrossEntropyLoss()
optimizer = torch.optim.SGD(list(fc1.parameters()) + list(fc2.parameters()), 1e-1)

# for _ in range(1000):
#     optimizer.zero_grad()
#     x = torch.Tensor([[1,2,3]])
#     xS = torch.zeros_like(x)
#     labels = torch.LongTensor([0])

#     x1, xS1 = fc1(x, xS)
#     x1.retain_grad()
#     xS1.retain_grad()
#     x2, xS2 = relu(x1, xS1)
#     x2.retain_grad()
#     xS2.retain_grad()
#     x3, xS3 = fc2(x2, xS2)
#     x3.retain_grad()
#     xS3.retain_grad()

#     loss = criteria(x3, xS3, labels)
#     loss.backward()
#     optimizer.step()

optimizer = torch.optim.SGD(list(fc1.parameters()) + list(fc2.parameters()), 0)
optimizer.zero_grad()
x = torch.Tensor([[1,2,3]])
# x = torch.Tensor([[1,2,3], [1,2,3]])
xS = torch.zeros_like(x)
labels = torch.LongTensor([0])
# labels = torch.LongTensor([0,0])

x1, xS1 = fc1(x, xS)
x1.retain_grad()
xS1.retain_grad()
x2, xS2 = relu(x1, xS1)
x2.retain_grad()
xS2.retain_grad()
x3, xS3 = fc2(x2, xS2)
x3.retain_grad()
xS3.retain_grad()

loss = criteria(x3, xS3, labels)
loss.backward()
optimizer.step()
print(loss)

tensor(0.0126, grad_fn=<SCrossEntropyLossFunctionBackward>)


In [3]:
# x3SG = torch.Tensor([[0.0123928,-0.0117341,-0.000658689,-0.0117341,0.011742,-7.92681E-6,-0.000658689,-7.92681E-6,0.000666616]]).view(1,3,3)
# x1SG = torch.Tensor([[0.0249031,0.00560088,0.00560088,0.00560088,0.00129391,0.00129391,0.00560088,0.00129391,0.00129391]]).view(1,3,3)

In [4]:
x3

tensor([[8.8600, 4.4400, 1.5600]], grad_fn=<SLinearFunctionBackward>)

In [5]:
def CECross(output):
    BS = output.shape[0]
    e3 = (output - output.max()).exp()
    e3_sum = e3.sum(dim=1)
    ratio = (e3 / e3_sum).view(BS,1,-1)
    return (torch.diag_embed(ratio.view(BS,-1),0,1) - ratio.swapaxes(1,2).bmm(ratio))
x3SG = CECross(x3)

In [6]:
x3SG

tensor([[[ 1.2393e-02, -1.1734e-02, -6.5869e-04],
         [-1.1734e-02,  1.1742e-02, -7.9268e-06],
         [-6.5869e-04, -7.9268e-06,  6.6662e-04]]], grad_fn=<SubBackward0>)

In [7]:
x1SG = fc2.op.weight.t().matmul(x3SG).matmul(fc2.op.weight)

In [8]:
x1SG

tensor([[[0.0249, 0.0056, 0.0056],
         [0.0056, 0.0013, 0.0013],
         [0.0056, 0.0013, 0.0013]]], grad_fn=<UnsafeViewBackward>)

In [9]:
def crossSecond(IN, outS):
    BS = IN.shape[0]
    IN = IN.view(BS,1,-1)
    return IN.swapaxes(1,2).bmm(IN).view(BS,-1).t().mm(outS.view(BS,-1))
fc1WSG = crossSecond(x,  x1SG)
fc2WSG = crossSecond(x2, x3SG)

In [19]:
fc1WSG.view(3,3,3,3).swapaxes(0,2).swapaxes(0,1).reshape(9,9)

tensor([[0.0249, 0.0056, 0.0056, 0.0498, 0.0112, 0.0112, 0.0747, 0.0168, 0.0168],
        [0.0056, 0.0013, 0.0013, 0.0112, 0.0026, 0.0026, 0.0168, 0.0039, 0.0039],
        [0.0056, 0.0013, 0.0013, 0.0112, 0.0026, 0.0026, 0.0168, 0.0039, 0.0039],
        [0.0498, 0.0112, 0.0112, 0.0996, 0.0224, 0.0224, 0.1494, 0.0336, 0.0336],
        [0.0112, 0.0026, 0.0026, 0.0224, 0.0052, 0.0052, 0.0336, 0.0078, 0.0078],
        [0.0112, 0.0026, 0.0026, 0.0224, 0.0052, 0.0052, 0.0336, 0.0078, 0.0078],
        [0.0747, 0.0168, 0.0168, 0.1494, 0.0336, 0.0336, 0.2241, 0.0504, 0.0504],
        [0.0168, 0.0039, 0.0039, 0.0336, 0.0078, 0.0078, 0.0504, 0.0116, 0.0116],
        [0.0168, 0.0039, 0.0039, 0.0336, 0.0078, 0.0078, 0.0504, 0.0116, 0.0116]],
       grad_fn=<UnsafeViewBackward>)

In [13]:
c = torch.zeros(9,9)
for i in range(9):
    for j in range(9):
        xa = i // 3
        xb = i % 3
        ya = j // 3
        yb = j % 3
        c[xa*3 + ya, xb * 3 + yb] = fc1WSG[i,j]
print(c)
print(c.diag())
print(c.max())

tensor([[0.0249, 0.0056, 0.0056, 0.0498, 0.0112, 0.0112, 0.0747, 0.0168, 0.0168],
        [0.0056, 0.0013, 0.0013, 0.0112, 0.0026, 0.0026, 0.0168, 0.0039, 0.0039],
        [0.0056, 0.0013, 0.0013, 0.0112, 0.0026, 0.0026, 0.0168, 0.0039, 0.0039],
        [0.0498, 0.0112, 0.0112, 0.0996, 0.0224, 0.0224, 0.1494, 0.0336, 0.0336],
        [0.0112, 0.0026, 0.0026, 0.0224, 0.0052, 0.0052, 0.0336, 0.0078, 0.0078],
        [0.0112, 0.0026, 0.0026, 0.0224, 0.0052, 0.0052, 0.0336, 0.0078, 0.0078],
        [0.0747, 0.0168, 0.0168, 0.1494, 0.0336, 0.0336, 0.2241, 0.0504, 0.0504],
        [0.0168, 0.0039, 0.0039, 0.0336, 0.0078, 0.0078, 0.0504, 0.0116, 0.0116],
        [0.0168, 0.0039, 0.0039, 0.0336, 0.0078, 0.0078, 0.0504, 0.0116, 0.0116]],
       grad_fn=<CopySlices>)
tensor([0.0249, 0.0013, 0.0013, 0.0996, 0.0052, 0.0052, 0.2241, 0.0116, 0.0116],
       grad_fn=<DiagBackward>)
tensor(0.2241, grad_fn=<MaxBackward1>)


In [14]:
fc1WSG

tensor([[0.0249, 0.0056, 0.0056, 0.0056, 0.0013, 0.0013, 0.0056, 0.0013, 0.0013],
        [0.0498, 0.0112, 0.0112, 0.0112, 0.0026, 0.0026, 0.0112, 0.0026, 0.0026],
        [0.0747, 0.0168, 0.0168, 0.0168, 0.0039, 0.0039, 0.0168, 0.0039, 0.0039],
        [0.0498, 0.0112, 0.0112, 0.0112, 0.0026, 0.0026, 0.0112, 0.0026, 0.0026],
        [0.0996, 0.0224, 0.0224, 0.0224, 0.0052, 0.0052, 0.0224, 0.0052, 0.0052],
        [0.1494, 0.0336, 0.0336, 0.0336, 0.0078, 0.0078, 0.0336, 0.0078, 0.0078],
        [0.0747, 0.0168, 0.0168, 0.0168, 0.0039, 0.0039, 0.0168, 0.0039, 0.0039],
        [0.1494, 0.0336, 0.0336, 0.0336, 0.0078, 0.0078, 0.0336, 0.0078, 0.0078],
        [0.2241, 0.0504, 0.0504, 0.0504, 0.0116, 0.0116, 0.0504, 0.0116, 0.0116]],
       grad_fn=<MmBackward>)

In [48]:
fc2WSG

tensor([[ 2.4290e-02, -2.2999e-02, -1.2910e-03, -2.2999e-02,  2.3014e-02,
         -1.5537e-05, -1.2910e-03, -1.5537e-05,  1.3066e-03],
        [ 5.5519e-02, -5.2569e-02, -2.9509e-03, -5.2569e-02,  5.2604e-02,
         -3.5512e-05, -2.9509e-03, -3.5512e-05,  2.9864e-03],
        [ 8.6749e-02, -8.2138e-02, -4.6108e-03, -8.2138e-02,  8.2194e-02,
         -5.5488e-05, -4.6108e-03, -5.5488e-05,  4.6663e-03],
        [ 5.5519e-02, -5.2569e-02, -2.9509e-03, -5.2569e-02,  5.2604e-02,
         -3.5512e-05, -2.9509e-03, -3.5512e-05,  2.9864e-03],
        [ 1.2690e-01, -1.2016e-01, -6.7450e-03, -1.2016e-01,  1.2024e-01,
         -8.1170e-05, -6.7450e-03, -8.1170e-05,  6.8261e-03],
        [ 1.9828e-01, -1.8774e-01, -1.0539e-02, -1.8774e-01,  1.8787e-01,
         -1.2683e-04, -1.0539e-02, -1.2683e-04,  1.0666e-02],
        [ 8.6749e-02, -8.2138e-02, -4.6108e-03, -8.2138e-02,  8.2194e-02,
         -5.5488e-05, -4.6108e-03, -5.5488e-05,  4.6663e-03],
        [ 1.9828e-01, -1.8774e-01, -1.0539e-02, 

In [10]:
fc1WSG.view(9,9).diag().view(-1).sort()[1]

tensor([4, 5, 7, 1, 3, 8, 6, 2, 0])

In [11]:
fc1.weightS.grad.view(-1).sort()[1]

tensor([6, 3, 7, 4, 0, 8, 5, 1, 2])

In [64]:
O = torch.ones(3,2,2)
I = torch.ones(3,10)
a = torch.nn.Linear(10,2)
a.weight.t().matmul(O).matmul(a.weight).shape

torch.Size([3, 10, 10])

In [65]:
I.mm(a.weight.t())

tensor([[-0.7404, -0.0510],
        [-0.7404, -0.0510],
        [-0.7404, -0.0510]], grad_fn=<MmBackward>)

In [1]:
from Cmodules import CLinear, CCrossEntropyLoss
import torch
fc1 = CLinear(3,3,bias=False)
fc1.op.weight.data = torch.Tensor(list(range(1,10))).view(fc1.op.weight.shape) * 0.1
# fc1.op.bias.data = torch.zeros_like(fc1.op.bias)
fc2 = CLinear(3,3,bias=False)
fc2.op.weight.data = torch.Tensor(list(range(9,0,-1))).view(fc2.op.weight.shape) * 0.1
fc2.op.weight.data[0,0] = 2
# fc2.op.bias.data = torch.zeros_like(fc2.op.bias)
relu = torch.nn.ReLU()
criteria = CCrossEntropyLoss()
optimizer = torch.optim.SGD(list(fc1.parameters()) + list(fc2.parameters()), 1e-1)

# for _ in range(1000):
#     optimizer.zero_grad()
#     x = torch.Tensor([[1,2,3]])
#     xS = torch.zeros_like(x)
#     labels = torch.LongTensor([0])

#     x1, xS1 = fc1(x, xS)
#     x1.retain_grad()
#     xS1.retain_grad()
#     x2, xS2 = relu(x1, xS1)
#     x2.retain_grad()
#     xS2.retain_grad()
#     x3, xS3 = fc2(x2, xS2)
#     x3.retain_grad()
#     xS3.retain_grad()

#     loss = criteria(x3, xS3, labels)
#     loss.backward()
#     optimizer.step()

optimizer = torch.optim.SGD(list(fc1.parameters()) + list(fc2.parameters()), 0)
optimizer.zero_grad()
x = torch.Tensor([[1,2,3]])
x = torch.Tensor([[1,2,3], [1,2,3]])
xS = torch.zeros_like(x)
labels = torch.LongTensor([0])
labels = torch.LongTensor([0,0])

x1 = fc1(x)
x2 = relu(x1)
x3 = fc2(x2)

loss = criteria(x3, labels)
loss.backward()
optimizer.step()
print(loss)

tensor(0.0126, grad_fn=<NllLossBackward>)


In [2]:
S3 = criteria.Cbackward(x3)
S1 = fc2.Cbackward(S3)
S0 = fc1.Cbackward(S1)

torch.Size([2, 3])


In [7]:
S1

tensor([[[0.0249, 0.0056, 0.0056],
         [0.0056, 0.0013, 0.0013],
         [0.0056, 0.0013, 0.0013]],

        [[0.0249, 0.0056, 0.0056],
         [0.0056, 0.0013, 0.0013],
         [0.0056, 0.0013, 0.0013]]], grad_fn=<UnsafeViewBackward>)

In [5]:
fc1.gradWS

tensor([[0.0498, 0.0112, 0.0112, 0.0112, 0.0026, 0.0026, 0.0112, 0.0026, 0.0026],
        [0.0996, 0.0224, 0.0224, 0.0224, 0.0052, 0.0052, 0.0224, 0.0052, 0.0052],
        [0.1494, 0.0336, 0.0336, 0.0336, 0.0078, 0.0078, 0.0336, 0.0078, 0.0078],
        [0.0996, 0.0224, 0.0224, 0.0224, 0.0052, 0.0052, 0.0224, 0.0052, 0.0052],
        [0.1992, 0.0448, 0.0448, 0.0448, 0.0104, 0.0104, 0.0448, 0.0104, 0.0104],
        [0.2988, 0.0672, 0.0672, 0.0672, 0.0155, 0.0155, 0.0672, 0.0155, 0.0155],
        [0.1494, 0.0336, 0.0336, 0.0336, 0.0078, 0.0078, 0.0336, 0.0078, 0.0078],
        [0.2988, 0.0672, 0.0672, 0.0672, 0.0155, 0.0155, 0.0672, 0.0155, 0.0155],
        [0.4483, 0.1008, 0.1008, 0.1008, 0.0233, 0.0233, 0.1008, 0.0233, 0.0233]],
       grad_fn=<MmBackward>)

In [6]:
fc1.weightS.grad

tensor([[0.0498, 0.0224, 0.0336],
        [0.0224, 0.0104, 0.0155],
        [0.0336, 0.0155, 0.0233]], grad_fn=<ViewBackward>)

In [18]:
a = torch.Tensor(2,3)
b = torch.Tensor(2).unsqueeze(1)
print(b)
b = b.expand_as(a)
print(b)
a / b

tensor([[2.3877e-38],
        [3.6013e-43]])
tensor([[2.3877e-38, 2.3877e-38, 2.3877e-38],
        [3.6013e-43, 3.6013e-43, 3.6013e-43]])


tensor([[0., 0., 0.],
        [0., 0., 0.]])

In [20]:
import torch
from torch import nn
from Cmodules import CMaxpool2D, CReLU
from Cmodels import CMLP3
model = CMLP3()
a = torch.randn(2,28,28)
labels = torch.LongTensor([2,5])
criteria = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
optimizer.zero_grad()
o = model(a)
loss = criteria(o, labels)
loss.backward()
optimizer.step()
# model.Cbackward(o)

In [48]:
fc = nn.Linear(2,3,bias=False)
fc.weight.data = torch.Tensor([[1,2],[2,3],[3,4]])
x = torch.Tensor([[1,2]])
y = fc(x)

In [49]:
def C_cross_entropy_back(output):
    with torch.no_grad():
        BS = output.shape[0]
        e3 = (output - output.max()).exp()
        e3_sum = e3.sum(dim=1).unsqueeze(1).expand_as(e3)
        ratio = (e3 / e3_sum).view(BS,1,-1)
        return (torch.diag_embed(ratio.view(BS,-1),0,1) - ratio.swapaxes(1,2).bmm(ratio))
oS = C_cross_entropy_back(y)

In [50]:
print(oS)
print(y)

tensor([[[ 0.0024, -0.0001, -0.0022],
         [-0.0001,  0.0451, -0.0450],
         [-0.0022, -0.0450,  0.0472]]])
tensor([[ 5.,  8., 11.]], grad_fn=<MmBackward>)


In [51]:
BS = x.shape[0]
IN = x.view(BS,1,-1)
G = IN.swapaxes(1,2).bmm(IN).view(BS,-1).t().mm(oS.view(BS,-1))

In [60]:
T = G.view(2,2,3,3).swapaxes(0,2).swapaxes(0,1).reshape(6,6).numpy()

In [63]:
for i in range(6):
    for j in range(6):
        print(f"{T[i,j]:3f} ",end="")
    print()

0.002350 -0.000111 -0.002239 0.004700 -0.000223 -0.004477 
-0.000111 0.045076 -0.044964 -0.000223 0.090151 -0.089928 
-0.002239 -0.044964 0.047203 -0.004477 -0.089928 0.094405 
0.004700 -0.000223 -0.004477 0.009400 -0.000446 -0.008955 
-0.000223 0.090151 -0.089928 -0.000446 0.180302 -0.179856 
-0.004477 -0.089928 0.094405 -0.008955 -0.179856 0.188811 
